In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
from scipy.io import wavfile
import librosa
import os

class BlockChaoticDWTCompressedSensing:
    def __init__(self, wavelet='db4', compression_ratio=0.5, chaotic_map='logistic'):
        self.wavelet = wavelet
        self.compression_ratio = compression_ratio
        self.chaotic_map = chaotic_map
        
    def generate_chaotic_vector(self, length, seed=0):
        """Generate chaotic sequence without storing large matrix"""
        np.random.seed(seed)
        
        if self.chaotic_map == 'logistic':
            # Logistic map
            sequence = []
            x = 0.1 + seed * 0.01
            r = 3.99
            for _ in range(length):
                x = r * x * (1 - x)
                sequence.append(2 * x - 1)  # Map to [-1, 1]
            return np.array(sequence)
            
        elif self.chaotic_map == 'tent':
            # Tent map
            sequence = []
            x = 0.1 + seed * 0.01
            for _ in range(length):
                if x < 0.5:
                    x = 2 * x
                else:
                    x = 2 * (1 - x)
                sequence.append(2 * x - 1)
            return np.array(sequence)
            
        elif self.chaotic_map == 'bernoulli':
            # Bernoulli-like from chaotic behavior
            sequence = []
            x = 0.1 + seed * 0.01
            r = 3.99
            for _ in range(length):
                x = r * x * (1 - x)
                sequence.append(1 if x > 0.5 else -1)
            return np.array(sequence)
            
        elif self.chaotic_map == 'chebyshev':
            # Chebyshev map
            sequence = []
            x = 0.1 + seed * 0.01
            k = 4
            for _ in range(length):
                x = np.cos(k * np.arccos(x))
                sequence.append(x)
            return np.array(sequence)
        
        else:
            return np.random.randn(length)
    
    def block_chaotic_compression(self, audio_data, level=4):
        """Block-based compression that never stores full matrix"""
        print(f"\n🎵 Applying {self.chaotic_map.upper()} Chaotic Compression...")
        
        # Use appropriate DWT level
        max_possible_level = pywt.dwt_max_level(len(audio_data), self.wavelet)
        level = min(level, max_possible_level, 4)  # Reduced level for stability
        
        print(f"   DWT level: {level}")
        coeffs = pywt.wavedec(audio_data, self.wavelet, level=level)
        coeffs_flat = np.concatenate(coeffs)
        N = len(coeffs_flat)
        M = int(N * self.compression_ratio)
        
        print(f"   Coefficients: {N} → {M} (Compression: {M/N:.1%})")
        
        # Block processing - generate chaotic vectors on the fly
        compressed_coeffs = np.zeros(M)
        block_size = 1000  # Process 1000 measurements at a time
        
        print("   Block compression progress:")
        for i in range(0, M, block_size):
            end_idx = min(i + block_size, M)
            current_block_size = end_idx - i
            
            # Generate chaotic vectors for this block
            block_measurements = np.zeros(current_block_size)
            
            for j in range(current_block_size):
                # Generate chaotic measurement vector for this row
                chaotic_vector = self.generate_chaotic_vector(N, seed=i+j)
                # Dot product with coefficients
                block_measurements[j] = np.dot(chaotic_vector, coeffs_flat)
            
            compressed_coeffs[i:end_idx] = block_measurements
            
            if (i // block_size) % 5 == 0:
                print(f"      {min(i+block_size, M)}/{M} measurements completed")
        
        return compressed_coeffs, coeffs, N, M
    
    def block_chaotic_reconstruction(self, compressed_coeffs, original_coeff_length, method='gradient'):
        """Reconstruct using block-based approach"""
        print(f"   Reconstruction using {method} method...")
        
        M = len(compressed_coeffs)
        N = original_coeff_length
        
        if method == 'gradient':
            # Gradient descent reconstruction
            x_reconstructed = np.zeros(N)
            learning_rate = 0.01
            num_iterations = 20
            
            for iteration in range(num_iterations):
                gradient = np.zeros(N)
                
                # Compute gradient in blocks
                block_size = 500
                for i in range(0, M, block_size):
                    end_idx = min(i + block_size, M)
                    current_block_size = end_idx - i
                    
                    for j in range(current_block_size):
                        chaotic_vector = self.generate_chaotic_vector(N, seed=i+j)
                        y_pred = np.dot(chaotic_vector, x_reconstructed)
                        error = compressed_coeffs[i+j] - y_pred
                        gradient += error * chaotic_vector
                
                # Update
                x_reconstructed += learning_rate * gradient / M
                
                if iteration % 5 == 0:
                    error_norm = np.linalg.norm(gradient) / M
                    print(f"      Iteration {iteration+1}/{num_iterations}, Error: {error_norm:.6f}")
        
        elif method == 'simple':
            # Simple pseudo-inverse approximation
            x_reconstructed = np.zeros(N)
            block_size = 1000
            
            for i in range(0, M, block_size):
                end_idx = min(i + block_size, M)
                current_block_size = end_idx - i
                
                # Generate block matrix temporarily
                block_matrix = np.zeros((current_block_size, N))
                for j in range(current_block_size):
                    block_matrix[j] = self.generate_chaotic_vector(N, seed=i+j)
                
                # Partial reconstruction
                block_measurements = compressed_coeffs[i:end_idx]
                block_reconstruction = np.linalg.lstsq(block_matrix, block_measurements, rcond=None)[0]
                x_reconstructed += block_reconstruction / (M // block_size)
        
        return x_reconstructed
    
    def reconstruct_audio(self, compressed_coeffs, original_coeff_length, level=4):
        """Full audio reconstruction pipeline"""
        # Reconstruct coefficients
        coeffs_reconstructed_flat = self.block_chaotic_reconstruction(
            compressed_coeffs, original_coeff_length, method='gradient'
        )
        
        # Reconstruct DWT coefficient structure
        coeffs_reconstructed = []
        start_idx = 0
        coeffs_slices = pywt.wavedec(np.zeros(original_coeff_length), self.wavelet, level=level)
        
        for coeff in coeffs_slices:
            end_idx = start_idx + len(coeff)
            if end_idx <= len(coeffs_reconstructed_flat):
                coeff_reconstructed = coeffs_reconstructed_flat[start_idx:end_idx]
                coeffs_reconstructed.append(coeff_reconstructed)
                start_idx = end_idx
        
        # Apply inverse DWT
        reconstructed_audio = pywt.waverec(coeffs_reconstructed, self.wavelet)
        
        return reconstructed_audio
    
    def calculate_metrics(self, original, reconstructed):
        """Calculate comprehensive quality metrics"""
        min_len = min(len(original), len(reconstructed))
        original_trimmed = original[:min_len]
        reconstructed_trimmed = reconstructed[:min_len]
        
        # MSE and SNR
        mse = np.mean((original_trimmed - reconstructed_trimmed)**2)
        snr = 10 * np.log10(np.var(original_trimmed) / (mse + 1e-10))
        
        # Additional metrics
        correlation = np.corrcoef(original_trimmed, reconstructed_trimmed)[0, 1]
        max_error = np.max(np.abs(original_trimmed - reconstructed_trimmed))
        
        return {
            'mse': mse,
            'snr': snr,
            'correlation': correlation,
            'max_error': max_error
        }

def efficient_chaotic_comparison(wav_file_path):
    """Memory-efficient comparison of chaotic maps"""
    
    if not os.path.exists(wav_file_path):
        print(f"❌ File '{wav_file_path}' not found!")
        return
    
    print(f"🎵 Loading: {wav_file_path}")
    
    # Load with shorter duration for stability
    audio_data, sample_rate = librosa.load(wav_file_path, sr=None, mono=True, duration=2)
    audio_data = audio_data / np.max(np.abs(audio_data))
    
    print(f"📊 Processing: {len(audio_data)} samples, {sample_rate} Hz")
    
    # Test chaotic maps
    chaotic_maps = ['logistic', 'tent', 'bernoulli', 'chebyshev']
    results = {}
    
    plt.figure(figsize=(15, 10))
    
    for idx, chaotic_map in enumerate(chaotic_maps):
        print(f"\n{'='*50}")
        print(f"🧪 Testing {chaotic_map.upper()}")
        print(f"{'='*50}")
        
        try:
            # Initialize compressor
            compressor = BlockChaoticDWTCompressedSensing(
                wavelet='db4',
                compression_ratio=0.5, 
                chaotic_map=chaotic_map
            )
            
            # Apply compression
            compressed_data, coeffs, N, M = compressor.block_chaotic_compression(audio_data)
            
            # Reconstruct
            reconstructed = compressor.reconstruct_audio(compressed_data, N)
            
            # Calculate metrics
            min_len = min(len(audio_data), len(reconstructed))
            metrics = compressor.calculate_metrics(
                audio_data[:min_len], 
                reconstructed[:min_len]
            )
            
            results[chaotic_map] = {
                'reconstructed': reconstructed[:min_len],
                'metrics': metrics,
                'compression_ratio': M/N
            }
            
            print(f"✅ {chaotic_map.upper()} Results:")
            print(f"   - SNR: {metrics['snr']:.2f} dB")
            print(f"   - MSE: {metrics['mse']:.6f}")
            print(f"   - Correlation: {metrics['correlation']:.4f}")
            print(f"   - Max Error: {metrics['max_error']:.6f}")
            
            # Plot time domain comparison
            plt.subplot(2, 2, idx+1)
            time_axis = np.arange(min_len) / sample_rate
            
            # Plot first second
            plot_samples = min(int(1.0 * sample_rate), min_len)
            plt.plot(time_axis[:plot_samples], audio_data[:plot_samples], 
                    'b-', alpha=0.8, label='Original', linewidth=1.5)
            plt.plot(time_axis[:plot_samples], reconstructed[:plot_samples], 
                    'r-', alpha=0.7, label='Reconstructed', linewidth=1)
            plt.title(f'{chaotic_map.upper()}\nSNR: {metrics["snr"]:.1f} dB')
            plt.xlabel('Time (s)')
            plt.ylabel('Amplitude')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
        except Exception as e:
            print(f"❌ Error with {chaotic_map}: {str(e)[:100]}...")
            results[chaotic_map] = None
    
    # Plot comparison summary
    successful_results = {k: v for k, v in results.items() if v is not None}
    
    if successful_results:
        # Metrics comparison plot
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
        
        maps = list(successful_results.keys())
        snr_values = [result['metrics']['snr'] for result in successful_results.values()]
        mse_values = [result['metrics']['mse'] for result in successful_results.values()]
        corr_values = [result['metrics']['correlation'] for result in successful_results.values()]
        max_err_values = [result['metrics']['max_error'] for result in successful_results.values()]
        
        # SNR comparison
        bars1 = ax1.bar(maps, snr_values, color='lightblue', alpha=0.7)
        ax1.set_title('SNR Comparison (Higher is Better)')
        ax1.set_ylabel('SNR (dB)')
        ax1.grid(True, alpha=0.3)
        for bar, value in zip(bars1, snr_values):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                    f'{value:.1f}', ha='center', va='bottom')
        
        # MSE comparison
        bars2 = ax2.bar(maps, mse_values, color='lightcoral', alpha=0.7)
        ax2.set_title('MSE Comparison (Lower is Better)')
        ax2.set_ylabel('MSE')
        ax2.grid(True, alpha=0.3)
        for bar, value in zip(bars2, mse_values):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.0001,
                    f'{value:.4f}', ha='center', va='bottom')
        
        # Correlation comparison
        bars3 = ax3.bar(maps, corr_values, color='lightgreen', alpha=0.7)
        ax3.set_title('Correlation Comparison (Higher is Better)')
        ax3.set_ylabel('Correlation Coefficient')
        ax3.set_ylim(0, 1)
        ax3.grid(True, alpha=0.3)
        for bar, value in zip(bars3, corr_values):
            ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                    f'{value:.3f}', ha='center', va='bottom')
        
        # Max Error comparison
        bars4 = ax4.bar(maps, max_err_values, color='lightyellow', alpha=0.7)
        ax4.set_title('Max Error Comparison (Lower is Better)')
        ax4.set_ylabel('Max Error')
        ax4.grid(True, alpha=0.3)
        for bar, value in zip(bars4, max_err_values):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                    f'{value:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        # Print results summary
        print(f"\n{'='*60}")
        print("📊 FINAL RESULTS SUMMARY")
        print(f"{'='*60}")
        
        best_snr = max(successful_results.items(), key=lambda x: x[1]['metrics']['snr'])
        best_mse = min(successful_results.items(), key=lambda x: x[1]['metrics']['mse'])
        
        print(f"🏆 BEST PERFORMANCE:")
        print(f"   Highest SNR: {best_snr[0].upper()} ({best_snr[1]['metrics']['snr']:.2f} dB)")
        print(f"   Lowest MSE:  {best_mse[0].upper()} ({best_mse[1]['metrics']['mse']:.6f})")
        
        print(f"\n📈 DETAILED RESULTS:")
        print(f"{'Map':<12} | {'SNR (dB)':<8} | {'MSE':<10} | {'Correlation':<10} | {'Max Error':<10}")
        print(f"{'-'*60}")
        for map_name, result in successful_results.items():
            metrics = result['metrics']
            print(f"{map_name.upper():<12} | {metrics['snr']:>8.2f} | {metrics['mse']:>10.6f} | "
                  f"{metrics['correlation']:>10.4f} | {metrics['max_error']:>10.6f}")
        
        # Save best result
        best_method = best_snr[0]
        best_audio = successful_results[best_method]['reconstructed']
        
        output_dir = 'chaotic_compression_results'
        os.makedirs(output_dir, exist_ok=True)
        original_filename = os.path.splitext(os.path.basename(wav_file_path))[0]
        
        wavfile.write(f'{output_dir}/{original_filename}_original.wav', sample_rate,
                     np.int16(audio_data * 32767))
        wavfile.write(f'{output_dir}/{original_filename}_{best_method}_best.wav', sample_rate,
                     np.int16(best_audio * 32767))
        
        print(f"\n💾 Saved best result ({best_method.upper()}) to '{output_dir}' folder")
    
    return results

# **🎯 MAIN EXECUTION 🎯**
if __name__ == "__main__":
    # ==============================================
    # 🔽 PUT YOUR WAV FILE PATH HERE 🔽
    # ==============================================
    YOUR_WAV_FILE_PATH = "UrbanSound8K/any_wav.wav"  # ⬅️ CHANGE THIS!
    # ==============================================
    
    print("🚀 Starting Memory-Efficient Chaotic Map Comparison...")
    print("💡 Using block processing to avoid memory issues...")
    
    results = efficient_chaotic_comparison(YOUR_WAV_FILE_PATH)
    
    print(f"\n✅ Chaotic Map Analysis Complete!")

🚀 Starting Memory-Efficient Chaotic Map Comparison...
💡 Using block processing to avoid memory issues...
🎵 Loading: UrbanSound8K/any_wav.wav
📊 Processing: 88200 samples, 44100 Hz

🧪 Testing LOGISTIC

🎵 Applying LOGISTIC Chaotic Compression...
   DWT level: 4
   Coefficients: 88227 → 44113 (Compression: 50.0%)
   Block compression progress:


C:\Users\Shyam\AppData\Local\Temp\ipykernel_17280\3461649354.py:95: RuntimeWarning: invalid value encountered in dot
  block_measurements[j] = np.dot(chaotic_vector, coeffs_flat)


      1000/44113 measurements completed
      6000/44113 measurements completed
      11000/44113 measurements completed
      16000/44113 measurements completed
      21000/44113 measurements completed
      26000/44113 measurements completed
      31000/44113 measurements completed
      36000/44113 measurements completed
      41000/44113 measurements completed
   Reconstruction using gradient method...


C:\Users\Shyam\AppData\Local\Temp\ipykernel_17280\3461649354.py:128: RuntimeWarning: invalid value encountered in dot
  y_pred = np.dot(chaotic_vector, x_reconstructed)


KeyboardInterrupt: 

<Figure size 1500x1000 with 0 Axes>